# Datalake I-94 Visits
### Data Engineering Capstone Project

#### Project Summary
The National Travel and Tourism Office (NTTO) manages the ADIS/I-94 visitor arrivals program in cooperation with the Department of Homeland Security (DHS)/U.S. Customs and Border Protection (CBP). The I-94 provides a count of visitor arrivals to the United States (with stays of 1-night or more and visiting under certain visa types) to calculate U.S. travel and tourism volume exports.

As a data engineering team, we want to build a I-94 data model for our analysis team to do statistical analysis like: Which has been the most visited city in US? For how many days? For what reasion. And how does it evolve? Is it related to other factors like temperature or population of the city?

We will build an ETL to firstly stage the raw dataset in our data lake, then we will build our fact and dimension tables based on the staging tables in the data lake.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import os
import configparser
from datetime import datetime, timedelta
import pandas as pd, re
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, dayofmonth, month, year, weekofyear, date_format

### Step 1: Scope the Project and Gather Data

#### Scope 
In this project, we will build an ETL to build staging tables, fact table and dimension tables in the data lake using tools like `Pandas`, `PySpark`, `AWS S3`, etc.
 And we will use three dataset: `I94 Immigration Data`, `City Temperature Data` and `U.S. City Demographic Data`.

#### Describe and Gather Data 

- **I94 immigration** [data](https://travel.trade.gov/research/reports/i94/historical/2016.html) comes from the US National Tourism and Trade Office. It is provided in . [sas7bdat](https://cran.r-project.org/web/packages/sas7bdat/vignettes/sas7bdat.pdf). The main attributes include:

 * `i94cit` = code for visitor origin country
 * `i94port` = code for destination USA city
 * `arrdate` = arrival date in the USA
 * `i94mode` = code for transportation mode
 * `depdate` = departure date from the USA
 * `i94visa` = code for visa type (reason for visiting)

- **City Temperature** [data](https://www.kaggle.com/sudalairajkumar/daily-temperature-of-major-cities) comes from Kaggle. It is provided in csv format. The major attributes include:
  * `Date` 
  * `AverageTemperature`
  * `City` 
  * `State`
  * `Country`
  
- **U.S. City Demographic** [data](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/) comes from Opendatasoft. It is provided in csv format. The major attributes include:
  * `City` 
  * `State` 
  * `Median Age`
  * `Male Population` 
  * `Female Population` 
  * `Total Population`
  * `Number of Veterans` 
  * `Foreign-born` 
  * `Average Household Size`
  * `Race` 
  * `Count`

## Datalake Config 
- Data input path
- Data output path
- AWS key and password if using AWS S3

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS','AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS','AWS_SECRET_ACCESS_KEY')

input_data = config.get('DATA','INPUT')
output_data = config.get('DATA','OUTPUT')

## Load dataset

In [3]:
# create spark session
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [4]:
# load i94 immigration dataset
df_spark_i94 = spark.read.format('com.github.saurfang.sas.spark')\
                .load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
df_spark_i94.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [5]:
## Load us-cities-demographics
df_spark_dem = spark.read.options(header='True',inferSchema='True',delimiter=';').csv("us-cities-demographics.csv")
df_spark_dem.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



In [6]:
# Load city_temperature
df_spark_temp = spark.read.options(header='True',inferSchema='True').csv("city_temperature.csv")
df_spark_temp.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- State: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- AvgTemperature: double (nullable = true)



### Step 2: Explore and Assess the Data
#### Explore the Data 
Data quality issues:
###### i94 immigration data
- Invalid destination cities: missing port code or ports of entry don't belong to US
- Missing values for departure date

###### US cities-demographics data
- Invalid columns names: whitespce, '-', better to be lowercase.
- Duplicate rows due to race counint columns which is not necessary

###### Cities temperatures
- We only want temperature info for destination cities in April 2016
- Better to use lowercase for columns city and state for joining later with other table

#### Cleaning Steps
Drop duplicates records for all the dataframes

###### i94 immigration data
- Remove records missing invalid port of entry code
- Remove records missing departure date

###### US cities-demographics data
- Fix invalid columns names
- Remove columns race & count

###### Cities temperatures
- Filter records in April 2016, for US port of entry
- Change columns city and state to lowercase

In [7]:
# Create dictionary of valid i94port codes
re_filter = re.compile(r'\'(.*)\'.*\'(.*)\'')
i94port_map = {}
with open('mappings/i94prtl_valid.txt') as f:
     for line in f:
         groups = re_filter.search(line)
         i94port_map[groups[1]]=[groups[2]]

In [8]:
# Clean i94 immigration data
df_spark_i94_clr = df_spark_i94.dropDuplicates()\
                    .na.drop(subset=["depdate"])\
                    .filter(df_spark_i94.i94port.isin(list(i94port_map.keys())))\
                    .na.drop(subset=["matflag"])
df_spark_i94_clr.show(5)

+------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
| cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|
+------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
| 474.0|2016.0|   4.0| 103.0| 103.0|    NEW|20545.0|    2.0|   null|20547.0|  25.0|    2.0|  1.0|20160401|    null| null|      G|      O|   null|      M| 1991.0|06292016|     F|  null|    VES|5.5410441233E10|91285|      WT|
|1508.0|2016.0|   4.0| 104.0| 104.0|    NYC|20545.0|    1.0|     NY|20552.0|  16.0|    2.0|  1.0|2016040

In [9]:
# Write to parquet partitioned by arrdate
df_spark_i94_clr.write.partitionBy("arrdate").parquet(os.path.join(output_data, "stage_i94_immigration"), mode="overwrite")

In [10]:
# Clean us-cities-demographics data
df_spark_dem = df_spark_dem.toDF(*(c.replace(' ', '_') for c in df_spark_dem.columns))
df_spark_dem = df_spark_dem.toDF(*(c.replace('-', '_') for c in df_spark_dem.columns))
df_spark_dem = df_spark_dem.toDF(*[c.lower() for c in df_spark_dem.columns])
# Remove race count columns
df_spark_dem_clr = df_spark_dem.drop("race", "count")\
                    .dropDuplicates()\
                    .na.drop(subset=["city","state"])\
                    .withColumn("city", lower(col("city")))\
                    .withColumn("state", lower(col("state")))
df_spark_dem_clr.show(5)

+-----------+----------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+
|       city|     state|median_age|male_population|female_population|total_population|number_of_veterans|foreign_born|average_household_size|state_code|
+-----------+----------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+
|clarksville| tennessee|      29.7|          75029|            74161|          149190|             20803|        8211|                  2.64|        TN|
|    gresham|    oregon|      36.7|          53866|            56716|          110582|              6326|       17860|                  2.67|        OR|
|  fullerton|california|      34.5|          69549|            71300|          140849|              5394|       43404|                  2.97|        CA|
|saint louis|  missouri|      35.0|         153026|           162659|          315

In [11]:
# Write us-cities-demographics to parquet 
df_spark_dem_clr.write.parquet(os.path.join(output_data, "stage_cities_demographics"), mode="overwrite")

In [12]:
# Clean cities_temperatures data
# Convert city and state name to lower case
df_spark_temp = df_spark_temp.toDF(*[c.lower() for c in df_spark_temp.columns])
df_spark_temp_clr = df_spark_temp.dropDuplicates()\
                                    .filter(df_spark_temp.country=="US")\
                                    .filter(df_spark_temp.year==2016)\
                                    .filter(df_spark_temp.month==4)\
                                    .withColumn("state", lower(col("state")))\
                                    .withColumn("city", lower(col("city")))
df_spark_temp_clr.show(5)

+-------------+-------+----------+----------+-----+---+----+--------------+
|       region|country|     state|      city|month|day|year|avgtemperature|
+-------------+-------+----------+----------+-----+---+----+--------------+
|North America|     US|    alaska| anchorage|    4| 22|2016|          49.2|
|North America|     US|    alaska| fairbanks|    4|  8|2016|          38.7|
|North America|     US|   arizona|    tucson|    4| 26|2016|          64.4|
|North America|     US|  arkansas|fort smith|    4|  7|2016|          62.2|
|North America|     US|california|    fresno|    4| 13|2016|          62.0|
+-------------+-------+----------+----------+-----+---+----+--------------+
only showing top 5 rows



In [13]:
# Write cities temperatures data to parquet
df_spark_temp_clr.write.parquet(os.path.join(output_data, "stage_uscities_temperatures"), mode="overwrite")

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
We will use star schema as below:
![Tux, the Linux mascot](schema_i94.png)

###### 3.1.1 Fact table
Our fact table **fact_i94visits** will contain informations from the i94 immigration data 
joined with daily average temperature on the port city and arrival date.

###### 3.1.2 Dimention tables
- `dim_us_ports` contains informations like US port of entry code, city, state code and state name.
- `dim_visa` maps visa type which gives information like reason for visiting.
- `dim_countries` tells which country does visitor come from.
- `dim_travelmode` gives mode of transportation: air, land or sea.
- `dim_demographics`contains median age and population informations about US port city.   
- `dim_date` contains date information like year, month, day, week of year and weekday.

#### 3.2 Mapping Out Data Pipelines
The pipeline steps are described below:
1. Load raw dataset from source into Spark dataframe: `df_spark_i94`,  `df_spark_dem` and `df_spark_temp` for one month.
2. Clean each Spark dataframe as decscibed in *Step 2 Cleaning steps* and write each cleaned dataframe into parquet as staging table: `stage_i94_immigration`, `stage_cities_demographics` and `stage_uscities_temperatures`.
3. Create and load dimension tables: `dim_us_ports`, `dim_visa`, `dim_countries`, `dim_travelmode` and `dim_demographics`.
4. Create and load fact table `fact_i94_visits` joining `stage_i94_immigration` and `stage_uscities_temperatures`.
5. Create and load dimension tables  and `dim_date`.
6. Quality checks

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [14]:
from etl import stage_i94, \
                stage_demographics, \
                stage_temperatures, \
                load_dim_port, \
                load_dim_country, \
                load_dim_visa, \
                load_dim_travelmode, \
                load_dim_demographics, \
                build_fact_i94visits, \
                load_dim_date, \
                check_count, \
                check_uniquekey

In [15]:
# Create Spark session
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

#### 4.1.1 Create staging datasets

In [16]:
# Load, Clean and Stage I94 immigration dataset
stage_i94(spark, \
          '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', \
          output_data, 'stage_i94_immigration').show(5)

stage_i94 start
stage_i94 end
+------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
| cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|
+------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
| 474.0|2016.0|   4.0| 103.0| 103.0|    NEW|20545.0|    2.0|   null|20547.0|  25.0|    2.0|  1.0|20160401|    null| null|      G|      O|   null|      M| 1991.0|06292016|     F|  null|    VES|5.5410441233E10|91285|      WT|
|1508.0|2016.0|   4.0| 104.0| 104.0|    NYC|20545.0|    1.0|     NY|20552.

In [17]:
# Load, Clean and Stage demographics dataset
stage_demographics(spark, \
          "us-cities-demographics.csv", \
          output_data, 'stage_cities_demographics').show(5)

stage_demographics start
stage_demographics end
+-----------+----------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+
|       city|     state|median_age|male_population|female_population|total_population|number_of_veterans|foreign_born|average_household_size|state_code|
+-----------+----------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+
|clarksville| tennessee|      29.7|          75029|            74161|          149190|             20803|        8211|                  2.64|        TN|
|    gresham|    oregon|      36.7|          53866|            56716|          110582|              6326|       17860|                  2.67|        OR|
|  fullerton|california|      34.5|          69549|            71300|          140849|              5394|       43404|                  2.97|        CA|
|saint louis|  missouri|      35.0

In [18]:
# Load, Clean and Stage temperatures dataset
stage_temperatures(spark, \
          "city_temperature.csv", \
          output_data, 'stage_uscities_temperatures').show(5)

stage_temperatures start
stage_temperatures end
+-------------+-------+----------+----------+-----+---+----+--------------+
|       region|country|     state|      city|month|day|year|avgtemperature|
+-------------+-------+----------+----------+-----+---+----+--------------+
|North America|     US|    alaska| anchorage|    4| 22|2016|          49.2|
|North America|     US|    alaska| fairbanks|    4|  8|2016|          38.7|
|North America|     US|   arizona|    tucson|    4| 26|2016|          64.4|
|North America|     US|  arkansas|fort smith|    4|  7|2016|          62.2|
|North America|     US|california|    fresno|    4| 13|2016|          62.0|
+-------------+-------+----------+----------+-----+---+----+--------------+
only showing top 5 rows



#### 4.1.2 Build dimension tables from mapping text

In [19]:
# Create and load dimension : dim_us_ports
load_dim_port(spark, output_data, 'dim_us_ports').show(5)

load_dim_port start
load_dim_port end
+----------+---------+---------+-------+
|state_code|port_code|     city|  state|
+----------+---------+---------+-------+
|        AZ|      DOU|  douglas|arizona|
|        AZ|      LUK|lukeville|arizona|
|        AZ|      NAC|     naco|arizona|
|        AZ|      NOG|  nogales|arizona|
|        AZ|      PHO|  phoenix|arizona|
+----------+---------+---------+-------+
only showing top 5 rows



In [20]:
# Create and load dimension : dim_countries
load_dim_country(spark, output_data, 'dim_countries').show(5)

load_dim_country start
load_dim_country end
+------------+--------------------+
|country_code|             country|
+------------+--------------------+
|         582|MEXICO Air Sea, a...|
|         236|         AFGHANISTAN|
|         101|             ALBANIA|
|         316|             ALGERIA|
|         102|             ANDORRA|
+------------+--------------------+
only showing top 5 rows



In [21]:
# Create and load dimension : dim_visa
load_dim_visa(spark, output_data, 'dim_us_visa').show(5)

load_dim_visa start
load_dim_visa end
+---------+--------+
|visa_code|    visa|
+---------+--------+
|        1|Business|
|        2|Pleasure|
|        3| Student|
+---------+--------+



In [22]:
# Create and load dimension : dim_travelmode
load_dim_travelmode(spark, output_data, 'dim_travelmode').show(5)

load_dim_travelmode start
load_dim_travelmode end
+---------+------------+
|mode_code|        mode|
+---------+------------+
|        1|         Air|
|        2|         Sea|
|        3|        Land|
|        9|Not reported|
+---------+------------+



In [23]:
# Create and load dimension : dim_demographics
load_dim_demographics(spark, output_data, 'dim_demographics').show(5)

load_dim_demographics start
load_dim_demographics end
+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+---------+
|median_age|male_population|female_population|total_population|number_of_veterans|foreign_born|average_household_size|state_code|port_code|
+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+---------+
|      33.8|         223960|           239915|          463875|             18572|       32016|                  2.15|        GA|      ATL|
|      31.4|         100135|           109673|          209808|              7288|       17735|                  2.36|        NY|      ROC|
|      37.3|          36850|            37165|           74015|              4312|       15365|                  2.45|        FL|      FMY|
|      34.1|         312237|           343523|          655760|             31189|       43318|           

In [24]:
#### 4.1.3 Build fact table

In [25]:
# Build fact table : dim_demographics
build_fact_i94visits(spark, output_data, 'fact_i94visits').show(5)

build_fact_i94visits start
build_fact_i94visits end
+---------+-----------+-----------+----+-------+------+-------+-------+------+--------------+
|    cicid|arrdate_iso|depdate_iso|stay|i94port|i94cit|i94mode|i94visa|i94bir|avgtemperature|
+---------+-----------+-----------+----+-------+------+-------+-------+------+--------------+
|5659207.0| 2016-04-30| 2016-05-08|   8|    NEW| 103.0|    1.0|    2.0|  55.0|          null|
|5659318.0| 2016-04-30| 2016-05-21|  21|    HOU| 103.0|    1.0|    2.0|  51.0|          76.7|
|5659356.0| 2016-04-30| 2016-05-04|   4|    NYC| 103.0|    1.0|    2.0|  29.0|          null|
|5659405.0| 2016-04-30| 2016-05-07|   7|    NYC| 103.0|    1.0|    2.0|  55.0|          null|
|5659433.0| 2016-04-30| 2016-05-08|   8|    NYC| 103.0|    1.0|    2.0|  46.0|          null|
+---------+-----------+-----------+----+-------+------+-------+-------+------+--------------+
only showing top 5 rows



In [26]:
#### 4.1.4 Build dimension table 

In [27]:
# Create Dimension date_table
load_dim_date(spark, output_data, 'dim_date').show(5)

load_dim_date start
load_dim_date end
+-----------+----+-----+---+----+-------+
|arrdate_iso|year|month|day|week|weekday|
+-----------+----+-----+---+----+-------+
| 2016-04-25|2016|    4| 25|  17|    Mon|
| 2016-04-22|2016|    4| 22|  16|    Fri|
| 2016-04-30|2016|    4| 30|  17|    Sat|
| 2016-04-26|2016|    4| 26|  17|    Tue|
| 2016-04-04|2016|    4|  4|  14|    Mon|
+-----------+----+-----+---+----+-------+
only showing top 5 rows



#### 4.2 Data Quality Checks
For data quality, we will checks records count and unique key of our final fact table

In [28]:
# Data quality check: Count records
check_count(spark, output_data, 'fact_i94visits', "i94 visits")

Quality check succes for i94 visits with 2746459 records.


In [29]:
# Data quality check: Unique key
check_uniquekey(spark, output_data, 'fact_i94visits', ['cicid'], "i94 visits")

Unique key check succes for i94 visits.


In [30]:
# Stop spark session
spark.stop()

#### 4.3 Data dictionary 

##### fact_i94visits
Fact table extracted from the I94 immigration data（from US National Tourism and Trade Office）joind with city temperature dataset（from kaggle)
* `cicid` = key id
* `arrdate_iso` = arrival date
* `depdate_iso` = depature date
* `stay` = number of days
* `i94port` = 3 character code of destination city
* `i94cit` = 3 digit code of origin country
* `i94mode` = 1 digit transportation mode code
* `i94visa` = 1 digit visa type code
* `i94bir` = Age of traveler (7 age groupings, mean and median)
* `avgtemperature` = average temperature in fahrenheit

#### dim_countries
Dimension table extracted from mapping text
* `country_code` = 3 digit code of origin country
* `country` = origin country name

#### dim_date
Dimension table extracted from fact_i94visits
* `arrdate_iso` = arrival date
* `year` = arrival year
* `month` = arrival month
* `day` = arrival day of month
* `week` = arrival week of year
* `weekday` = arrival weekday

#### dim_demographics
Dimension table extracted from U.S. City Demographic Data (OpenSoft)
* `port_code` = US port city code
* `state_code` = US state code
* `median_age` = median age
* `male_population` = male population
* `female_population` = female population
* `total_population` = total population
* `number_of_veterans` = number of veterans
* `foreign_born` = number of foreign born
* `average_household_size` = average household size

#### dim_travelmode
Dimension table extracted from mapping text
* `mode_code` = 1 digit code of transportation mode
* `mode` = Mode of transportation (air, land, sea)

#### dim_us_ports
Dimension table extracted from mapping text
* `state_code` = 1 digit code of transportation mode
* `port_code` = 1 digit code of transportation mode
* `city` = 1 digit code of transportation mode
* `state` = 1 digit code of transportation mode

#### dim_us_visa
Dimension table extracted from mapping text
* `visa_code` = 1 digit code of visa type
* `visa` = Type of visa (business, pleasure, student)

#### Step 5: Complete Project Write Up
* Rationale for the choice of tools and technologies for the project.

  - In this project, we have used pandas, Apache Spark and AWS S3 to build ETL pipline, data model and datalake. Pandas is good for dealing small dataset like the mapping text files. And once we start to process large volume dataset like i94 immigration data which contains over 3 million rows per month, we use Apache Spark, cause it executes much faster by caching data in memory across multiple parallel operations. Finally, to implement our datalake, we use AWS S3 to store our staging tables, fact and dimension tables. 
  

* How often the data should be updated and why.

    - It depends on how often the raw data be updated from the source. In our case, the I94 immigration data is updated monthly from the source, so it's better to run the ETL monthly. 
    

* How to approach the problem differently under the following scenarios:
 * The data was increased by 100x.
    - If the data was increased by 100 times, we may need a Spark cluster setup on AWS EMR or other cluster computing provider.


 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
    - We may use a scheduler tool for data pipline to manage the regular jobs, airflow will be a good solution.


 * The database needed to be accessed by 100+ people.
    - We may need a powerful resource manager which allows us to share multiple applications through a common resource.
      We could use Apache Yarn to increase the system efficiency.